<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/03-Pandas/A1-Introduction_to_Pandas_with_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pandas

## Setup and preliminaries

We start this notebook by typing a "magic" command that allows iPython notebooks to display plots directly in the browser.

In [ ]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [ ]:
# Render our plots inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In order to read and process files, we are going to use a very powerful, and widely used Python library, called pandas. So, our next step is to import the pandas library in Python, and also import the library matplotlib for generating plots:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

And we type some code to simply change the visual style of the plots. (The code below is optional and not necessary, and for now you do not need to understand what exactly is happening.)

In [ ]:
# Make the graphs a bit bigger
matplotlib.style.use(["seaborn-talk", "seaborn-ticks", "seaborn-whitegrid"])

# Loading Data

## From CSV Files

We will use a dataset with [restaurant inspection results in NYC](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j).

We fetch it by executing the following command:

In [ ]:
import os
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015', 
    db = 'doh_restaurants',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

We want to be able to read and process this file within Python. The pandas library has a very convenient method `read_csv` which reads the file, and returns back a variable that contains its contents.

In [ ]:
import pandas as pd


# This query returns back the restaurants in the DOH database

sql = '''
	SELECT R.CAMIS, R.DBA, R.BUILDING, R.STREET, R.ZIPCODE, R.BORO, 
          R.CUISINE_DESCRIPTION, R.LATITUDE, R.LONGITUDE
		FROM restaurants R

'''

restaurants = pd.read_sql(sql, con=con)

In [ ]:
# This query returns back the results and violations captured in the 
# latest inspection of each restaurant

sql = '''
  WITH latest_inspection AS (
		SELECT CAMIS, MAX(INSPECTION_DATE) AS INSPECTION_DATE FROM inspections
		GROUP BY CAMIS
	)
	SELECT R.CAMIS, R.DBA, R.ZIPCODE, R.BORO,
          I.INSPECTION_DATE, I.INSPECTION_ID, I.INSPECTION_TYPE, 
          V.VIOLATION_CODE, I.SCORE, I.GRADE
		FROM restaurants R
			JOIN latest_inspection L ON R.CAMIS = L.CAMIS
			JOIN inspections I ON I.CAMIS = L.CAMIS AND L.INSPECTION_DATE = I.INSPECTION_DATE
			JOIN violations V ON I.INSPECTION_ID = V.INSPECTION_ID
'''

inspections = pd.read_sql(sql, con=con)

When you run your query using Pandas, you get back a kind of object called a DataFrame, which is made up of rows and columns. Let's take a look at how the object looks like:

In [ ]:
restaurants.head(5)

In [ ]:
inspections.head(5)

# Data Types and Conversions

## Data Types using `.dtypes`

We can also check the data types for each column

In [ ]:
restaurants.dtypes

### Exercise

Get the data types for the "inspections"

In [ ]:
# your code here

## Data Types using `.describe()`

We can use the method "describe()" to get a quick overview of the data in the dataframe.

In [ ]:
restaurants.describe(include='all')

In [ ]:
# Same as above, but the .T command transposes the table
restaurants.describe(include='all').T

### Exercise

Analyze the "inspections" dataframe using the `.describe()` command

In [ ]:
# your code here

## Descriptive statistics



### Descriptive Statistics for Numeric Variables


#### Basic descriptive statistics for numeric variables

Given that SCORE is a numeric variable, we can get more detailed descriptive statistics for the variable using the `.describe()` command:

In [ ]:
inspections["SCORE"].describe()

And now that SCORE is a numeric variable, we can examine its distribution by using the `hist` command of Pandas, which creates a histogram. (The histogram is lso available as `plot.hist`, or `plot(kind='hist'))`.)

In [ ]:
inspections["SCORE"].hist()

By default, the histogram has ~10 bars in out plot. We can change the resolution of the histogram using the `bins` variable. Larger number of `bins` allow for higher resolution, but if we increase the number too much, many bins end up having very few, or no data points. For example, experiment with changing the balue of bins below, and change the value from 50 to something bigger.

In [ ]:
inspections["SCORE"].hist(bins=50)

In [ ]:
# A quick exposure to various options of the "hist" command
inspections["SCORE"].hist(
    bins=50,  # use 50 bars
    range=(0, 50),  # x-axis from 0 to 50
    density=True,  # show normalized count (density=True), or raw counts (density= False)
    figsize=(15, 5),  # controls the size of the plot
    alpha=0.8,  # make the plot 20% transparent
)

#### Kernel Density Estimation (KDE)

An alternative to histograms is to use the **kernel density**, which estimates a continuous function, instead of the bucketized counts, which tends to be discontunuous and bumpy. We can access this usind the `.plot(kind='kde')` command:

In [ ]:
# This plots the density of a numeric attribute
# kde = kernel density estimation
inspections["SCORE"].plot(kind="kde", color="Black", xlim=(0, 50), figsize=(15, 5))

#### Extra: Combining plots

Just as a quick note, we can actually easily combine plots in Pandas, by saving a plot in a variable, and then passing the parameter `ax = ....` in the new figure that we are creating. That `ax` parameter indicates that we want to plot the new chart on top of the plot specified in the `ax` parameter.

In [ ]:
# We save the plot in the variable "histogram_figure"
histogram_figure = inspections["SCORE"].hist(
    bins=50,  # use 50 bars
    range=(0, 50),  # x-axis from 0 to 50
    density=True,  # show normalized count (density=True), or raw counts (density= False)
    figsize=(15, 5),  # controls the size of the plot
    alpha=0.8,  # make the plot 20% transparent
)

# To combine the plots, we pass the ax = histogram_figure parameter
inspections["SCORE"].plot(
    kind="kde", color="Black", xlim=(0, 75), figsize=(15, 5), ax=histogram_figure
)

### Descriptive Statistics for Dates


In [ ]:
inspections["INSPECTION_DATE"].describe(datetime_is_numeric=True)

#### Exercise

* Plot a histogram for `INSPECTION_DATE`. What do you see?
* Try modifying the `bins` parameter. What are the results?
* The `range=(start, finish)` command is often useful, when we want to focus on a particular part of the dataset. Try using that for  `INSPECTION DATE` to limit the dates to be between 1/1/2019 and 05/30/2022.

In [ ]:
# your code here

#### Solution

In [ ]:
# Not very appealing
inspections["INSPECTION_DATE"].hist()

In [ ]:
inspections["INSPECTION_DATE"].hist(bins=50)

In [ ]:
x_start = pd.to_datetime("2019-01-01")
x_end = pd.to_datetime("2022-05-31")
bins = (x_end - x_start) / np.timedelta64(1, "M")  # one bin per month

inspections["INSPECTION_DATE"].hist(
    range=(
        x_start,
        x_end,
    ),  # limit the range of dates, ignore the 1/1/1990 faulty valye
    bins=round(bins),  # number of months in the range -- computed manually
    figsize=(15, 5),  # resize 15-width, 5-height
)

### Descriptive Statistics for Categorical/string columns

We can also get quick statistics about the common values that appear in each column:

In [ ]:
restaurants["DBA"].value_counts()

In [ ]:
restaurants["CUISINE_DESCRIPTION"].value_counts()

And we can use the "plot" command to plot the resulting histogram. (More details at http://pandas.pydata.org/pandas-docs/stable/visualization.html) We also revisit plotting below.

In [ ]:
restaurants["CUISINE_DESCRIPTION"].value_counts()[:20].plot(kind="barh")

In [ ]:
popular = restaurants["CUISINE_DESCRIPTION"].value_counts()
popular

In [ ]:
popular[:20].plot(kind="barh")

In [ ]:
# We can invert the list by starting at element 19 (the 20th element)
# and then go to the beginning of the list going backwards (the :-1 at the end)
popular[19::-1].plot(kind="barh")

#### Exercise

Using the "inspections" dataframe, analyze the `VIOLATION_CODE` attribute and identify the most common violation codes in the data. Create a bar plot as above to show the frequency of the various violations codes.

If you are adventurous, issue a query against the database using the `read_sql` command and get the descriptions of the violations from the table `doh_restaurants.violaction_codes`

In [ ]:
# your code here

#### Exercise

* Create a plot showing the number of restaurants across **boroughs**. Use the `BORO` column and the `value_counts()` function.

In [ ]:
# your code here

#### Solution 

In [ ]:
restaurants["BORO"].value_counts()

In [ ]:
restaurants["BORO"].value_counts().plot(kind="bar")

In [ ]:
restaurants["BORO"].value_counts().plot(kind="barh")

# Basic Data Manipulation Techniques

## Selecting a subset of the columns -- `filter()`

In a dataframe, we can specify the column(s) that we want to keep, and get back another dataframe with just the subset of the columns that we want to keep.

In [ ]:
inspections

In [ ]:
inspections.filter( 
    items = ["DBA", "GRADE", "INSPECTION_DATE"] 
)

In [ ]:
columns = ["CAMIS", "DBA", "GRADE", "INSPECTION_DATE", "SCORE"] 

# Notice the use of "chain notation" below
# Chain notation means putting parentheses around
# the command and then having each operation in its
# own line
(
  inspections
  .filter( items = columns )
  .head(10)
)


We can also use the `like` option to find all the column names that include a certain string. For example, to get all the columns that include the string `DATE`:

In [ ]:
inspections.filter(
    like = 'DATE'
)

We can expand the functionality and also use regular expressions:

In [ ]:
restaurants.filter(
    regex = r'^C' # all the columns that start with C
)

### Exercise

Keep the columns "DBA", "SCORE", "CUISINE DESCRIPTION" and "ZIPCODE".

In [ ]:
# your code here

## Renaming Columns -- `rename()`

To do the equivalent of `SELECT attr AS alias` in Pandas, we use the `rename` command, and pass a dictionary specifying which columns we want to rename:



In [ ]:
restaurants.rename(
    columns = {
      "CAMIS": "RESTID",
      "DBA": "REST_NAME",
      "BUILDING": "STREET_NUM",
      "BORO": "BOROUGH"
    }
)

## Selecting rows -- `query()`

To select rows, we can use the following approach, where we generate a list of boolean values, one for each row of the dataframe, and then we use the list to select which of the rows of the dataframe we want to keep"

In [ ]:
# Find all violations for restaurants with DBA being Starbucks
restaurants.query(' DBA == "STARBUCKS" ')

In [ ]:
# Find all violations with code 04L (i.e., "has mice")
inspections.query(' VIOLATION_CODE == "04L" ')

In [ ]:
# We can store the result in a dataframe called  has_mice
has_mice = inspections.query(' VIOLATION_CODE == "04L" ')
has_mice

In [ ]:
# The most frequent DBA names overall across restaurants
restaurants["DBA"].value_counts()[:20]

In [ ]:
# List the most frequent DBA values in the has_mice dataframe
has_mice["DBA"].value_counts()[:20]

And we can use more complex conditions. 

In [ ]:
has_mice_10012 = (
    inspections
    .query('  VIOLATION_CODE == "04L" and ZIPCODE == "10012" ')
    .filter( items = ['DBA', 'INSPECTION_DATE'])
)

has_mice_10012

In [ ]:
restaurants

## Selecting distinct values -- `drop_duplicates()`

We can do the equivalent of `SELECT DISTINCT` in Pandas by doing the following

In [ ]:
(
    restaurants
    .query(' CUISINE_DESCRIPTION == "Coffee/Tea"  and ZIPCODE == "10012" ')
    .filter( items = ['DBA'])
    .drop_duplicates()
)

## Sorting values -- `sort_values()`

And we can do the equivalent of `ORDER BY` by using the `.sort_values()

In [ ]:
(
    has_mice_10012
    .sort_values("INSPECTION_DATE", ascending=False)
    .head(15)
)

In [ ]:
(
    has_mice_10012
    .sort_values(["INSPECTION_DATE","DBA"], ascending=[False,True])
    .head(15)
)

## Defining New Columns -- `assign()` and `apply()`



### Using the `assign()` approach

The `assign` command applies a function to a dataframe and returns back a new dataframe with the new column(s).

In [ ]:
import numpy as np

# We define a function that will take as input a dataframe df
# and returns back a new column. This function computes
# the distance (in miles) from NYU, given the lat/lon of the 
# other location
def distance(df):
  NYU_lon = -73.9962293
  NYU_lat = 40.7291527
  # The calculation below is simply the Pythagorean theorem.
  # The normalizing values are just for converting lat/lon differences
  # to miles
  distance = ((df.LATITUDE-NYU_lat)/0.0146)**2 + ((df.LONGITUDE-NYU_lon)/0.0196)**2
  return np.sqrt(distance)

# This function combines STREET/BUILDING/BORO/ZIPCODE columns into one address
def combine_address(df):
  return (df.BUILDING + ' ' + df.STREET + ', ' + df.BORO + ', NY ' + df.ZIPCODE).str.upper()

In [ ]:
# We now use the `assign` function to create two new columns
# using the logic in the functions above,
(
  restaurants
  .assign(
      distance_from_NYU = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_NYU'])
)

In [ ]:
# And let's eliminate now duplicates and sort by distance
(
  restaurants
  .assign(
      distance_from_NYU = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_NYU'])
  .query('distance_from_NYU > 0') # eliminates NaN values from distance_from_NYU
  .drop_duplicates()
  .sort_values('distance_from_NYU')
  .head(25)
)

### Using the `apply` approach

The `apply` function allow the users to pass a function and apply it on every single row or column of a Pandas datarame. 

In [ ]:
!sudo pip3 install -q -U geopy

from geopy import distance

# A bit more accurate distance calculation, which returns back
# the distance in miles. However, we cannot pass a dataframe
# to the function but only individual values
def distance_from_NYU_geodesic(row):
  NYU_lon = -73.9962293
  NYU_lat = 40.7291527
  NYU = (NYU_lat, NYU_lon)
  rest = (row.LATITUDE, row.LONGITUDE)
  #if pd.isnull(row.Latitude) or pd.isnull(row.Longitude):
  #  return None
  return distance.distance(NYU, rest).miles


In [ ]:
# We now create a smaller version of the dataset with just
# the names/address/lon/lat of the restaurants
rest_names_locations = (
    restaurants
    .assign(
      address = combine_address
    )
    .filter(items = ['CAMIS','DBA','address','LONGITUDE', 'LATITUDE'])
    .query(' LONGITUDE==LONGITUDE ') # idiomatic expression for saying IS NOT NULL
    .query(' LATITUDE==LATITUDE ') # idiomatic expression for saying IS NOT NULL
    .drop_duplicates()
)

rest_names_locations

In [ ]:
# We will now apply the function distance_from_NYU_geodesic 
# to every row of the dataset:
rest_names_locations.apply(distance_from_NYU_geodesic, axis='columns')


In [ ]:
# We will now save the result into a new column
rest_names_locations['distance_from_NYU']=rest_names_locations.apply(distance_from_NYU_geodesic, axis='columns')

In [ ]:
# Let's see how many restaurants are within half a mile from NYU :)
(
    rest_names_locations
    .query('distance_from_NYU < 0.5')
    .sort_values('distance_from_NYU')
)

## Aggregation Function -- `agg()`

In [ ]:
inspections['SCORE'].agg('mean')

In [ ]:
inspections['SCORE'].agg(['mean','std','count','nunique'])

In [ ]:
inspections.agg(
    {
        'SCORE': ['mean','std','count','nunique'],
        'CAMIS':  ['nunique','count']
     }
    )

In [ ]:
inspections.agg(
        num_scored_violations = ('SCORE', 'count'),
        mean_score = ('SCORE', 'mean'),
        std_score  = ('SCORE', 'std'),
        num_entries = ('CAMIS',  'count'),
        num_restaurants = ('CAMIS',  'nunique'),
  )

## Calculating aggegates per groups -- `groupby()`

In [ ]:
inspections.groupby('INSPECTION_DATE').agg({'SCORE': 'mean'})

In [ ]:
(
  inspections
  .groupby('INSPECTION_DATE')
  .agg(
      score_mean = ('SCORE', 'mean'), 
      graded_restaurants = ('CAMIS', 'nunique')
    )
  .tail(500)
  .head(20)
)

In [ ]:
(
  inspections
  .groupby('INSPECTION_DATE')
  .agg(
      score_mean = ('SCORE', 'mean'), # calculate the aveage score for the date
      graded_restaurants = ('CAMIS', 'nunique') # and the number of restaurants
    )
  .query('graded_restaurants>10') # keep only days with at least 10 graded restauranta
  .filter(items=['score_mean']) # we only want to plot the score
  .plot()
)

## Pivot Tables

[Pivot tables](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) is one of the most commonly used exploratory tools, and in Pandas they are extremely flexible. 

For example, let's try to count the number of restaurants that are inspected every day. 

In [ ]:
# Count the number of CAMIS values that appear on each date

pivot = pd.pivot_table(
    data=inspections,
    index="INSPECTION_DATE",  # specifies the rows
    values="CAMIS",  # specifies the content of the cells
    aggfunc="count",  # we ask to count how many different CAMIS values we see
)

In [ ]:
pivot

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
pivot.plot(figsize=(10, 5))

In [ ]:
pivot[200:300].plot()

#### Changing date granularity 

We can also use the [resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) command to change the frequency from one day, to, say, 7 days. Then we can compute, say, the average (`mean()`) for these days, or the total number (`sum()`) of inspections.

In [ ]:
pivot.resample("1W").sum().tail(100)

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
# Plot the average number of inspections, over 7-day periods
pivot.resample("1W").mean().plot()

In [ ]:
# Plot the total number of inspections, over 1-month periods
pivot.resample("1M").sum().plot()

In [ ]:
plot = pivot.resample("7D").mean().plot()
plot.set_xlabel("Date of Inspection")
plot.set_ylabel("Average Number of Inspections (7-day average)")
plot.set_title("Analysis of Number of Inspections over Time")

#### Pivot Table with two (or more) variables)

We would like to break down the results by borough, so we add the `column` parameter.

In [ ]:
pivot2 = pd.pivot_table(
    data=inspections,  #
    index="INSPECTION_DATE",
    columns="BORO",
    values="CAMIS",
    aggfunc="count",
)
pivot2.head(10)

In [ ]:
pivot2.tail(5)

In [ ]:
pivot2.plot()

In [ ]:
# We resample the index, to keep only dates every one month
# For that one month period, we compute the average value
pivot2.resample("1M").mean().tail(10)

In [ ]:
# If we plot a dataframe then by default the index column
# in our case the INSPECTION DATE becomes the x-asis
# and all the columns become separate lines in the plot
pivot2.resample("1M").mean().plot()

# Potential Exercise, if we have time:
# Drop the last date, which corresponds to an incomplete month

## (Optional, FYI) Advanced Pivot Tables

We can also add multiple attributes in the index and columns. It is also possible to have multiple aggregation functions, and we can even define our own aggregation functions.

In [ ]:
# We write a function that returns the
# number of unique items in a list x
def count_unique(x):
    return len(set(x))


# We break down by BORO and GRADE, and also calculate
# inspections in unique (unique restaurants)
# and non-unique entries (effectuvely, violations)
pivot_advanced = pd.pivot_table(
    data=restaurants,  #
    index="GRADE DATE",
    columns=["BORO", "GRADE"],
    values="CAMIS",
    aggfunc=["count", count_unique],
)

# Take the total number of inspections (unique and non-unique)
agg = pivot_advanced.resample("1M").sum()

# Show the last 5 entries and show the transpose (.T)
agg.tail().T

### Exercise 1 

Now let's do the same exercise, but instead of counting the number of inspections, we want to compute the average score assigned by the inspectors. Hint: We will need to change the `values` and the `aggfunc` parameters in the `pivot_table` function above.

In [ ]:
# your code here

#### Solution

In [ ]:
pivot = pd.pivot_table(
    data=restaurants,
    index="INSPECTION DATE",  # specifies the rows
    values="SCORE",  # specifies the content of the cells
    aggfunc="mean",  # compute the average SCORE
)

In [ ]:
pivot.plot(figsize=(10, 4))

In [ ]:
pivot.resample("1W").mean().plot(figsize=(10, 4))



### Exercise 2

We now want to examine if different cuisines have different inspection scores. Compute the average inspection score by cuisine. Use the `sort_values()` command ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)) to order cuisines by their inspection scores.

In [ ]:
# your code here

#### Solution

In [ ]:
pivot = pd.pivot_table(
    data=restaurants,
    index="INSPECTION DATE",  # dates are the rows
    columns="CUISINE DESCRIPTION",  # cuisines are the columns
    values="SCORE",  # we analyze the SCORE
    aggfunc="mean",  # compute the average SCORE
)
# Select 3 columns (american, frenchn, chinese), compute the average score for a month
# and plot the results
pivot[["American", "French", "Chinese"]].resample("1M").mean().plot(figsize=(10, 4))